## Migration from Text Completions API to the new Messages API

The text completions API for Claude is now in legacy mode and it is recommended to migrate to 
the new Messages API. The Messages API is backward compatible with current Claude Models on Bedrock and requires that a modified JSON body be passed to the invoke methods. An example of a text completion API style body and prompt is shown below:

prompt='Human:\n\n What is quantum mechanics? \n\n Assistant:\n\n'

body = json.dumps({
                "prompt": prompt,
                "temperature": 0.5,
                "top_p": 0.9,
                "max_tokens_to_sample": 512,
            })

The new body will have the following form with an updated messages element.

body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 512,
            "temperature": 0.5,
            "top_p": 0.9,
            "system": system_prompt,
            "messages": messages
        }  
    ) 
    
The messages element represents a list of JSONL objects that include the role and content for each turn in an input. For a single input, the messages element is shown below where the "Human" tag from the text Completion API is replaced by "user" in the new API:

messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

Lets see how this works with some examples with Claude 2.1

### Initialize packages

In [2]:
%pip install --upgrade pip
%pip install boto3==1.33.2 --force-reinstall --quiet
%pip install botocore==1.33.2 --force-reinstall --quiet


Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.33.13 which is incompatible.
amazon-sagemaker-jupyter-scheduler 3.0.6 requires pydantic==1.*, but you have pydantic 2.6.3 which is incompatible.
jupyter-scheduler 2.4.0 requires sqlalchemy~=1.0, but you have sqlalchemy 2.0.27 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fo

#### Restart the kernel with the updated packages that are installed through the dependencies above

In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [15]:
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime')


### Helper function to pass our models, messages, and inference parameters

In [13]:
def generate_message(bedrock_runtime, model_id, messages, max_tokens,top_p,temp):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "anthropic_beta": ["private-messages-testing"],
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p
        }  
    )  
    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())

    return response_body

### Single-Input

In [14]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

generate_message(bedrock_client, model_id = 'anthropic.claude-v2:1',messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

{'id': 'compl_01XYQsypYjS8g1LSCL9PB142',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': 'Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the subatomic level. Some key points about quantum mechanics:\n\n- It explains phenomena that classical physics cannot, such as the wave-particle duality of matter and quantum entanglement. At the quantum scale, particles can behave like waves and vice versa.\n\n- It introduces concepts like quantum superposition and quantum uncertainty. Particles do not have defined properties until they are measured, only probabilities that collapse into a definite state upon measurement.\n\n- It is governed by the Schrödinger equation, which describes how the quantum state of a physical system changes over time. This equation has wavefunction solutions.\n\n- In the quantum realm, discreet energy levels exist, as opposed to the continuous energy spectrum found in classical m

### Multi-turn Input

In [7]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics? "}]},\
         { "role":'assistant', "content":[{'type':'text','text': "It is a branch of physics that \
         describes how matter and energy interact with discrete energy values "}]},\
         { "role":'user', "content":[{'type':'text','text': "Can you explain a bit more about discrete energies?"}]}]

generate_message(bedrock_client, model_id = 'anthropic.claude-v2:1',messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

{'id': 'compl_01W1wLhocC7qiXd8vbLwN8mV',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': 'Sure, the key idea around discrete energies in quantum mechanics is:\n\n- Matter and energy at the subatomic level exist only in specific, discrete amounts or "packets" - they do not exist in a continuous range of values. For example:\n\n- Electrons orbiting an atom can only exist at specific discrete energy levels. They can\'t have arbitrary in-between energies. They have to "jump" between allowed energy levels.\n\n- Photons (particles of light) carry packets or bundles of energy called quanta. The energy of a photon depends on its frequency but can only have certain values - it\'s quantized.\n\n- Atoms and molecules can only exist in states with specific discrete amounts of energy. They can gain/lose energy by absorbing/emitting photons that carry quanta of energy.\n\n- Particles like electrons can also act like waves. The wavelengths of these matter waves is r